In [20]:
!pip install -r requirements.txt

In [21]:
import os
import io
import sys
import json
import glob
import boto3
import base64
import logging
import requests
import botocore
import numpy as np
import pandas as pd
import globals as g
from PIL import Image
from typing import List
from pathlib import Path
from utils import get_cfn_outputs
from urllib.parse import urlparse
from botocore.auth import SigV4Auth
from sagemaker import get_execution_role
from botocore.awsrequest import AWSRequest
from pandas.core.series import Series
from sagemaker.huggingface.model import HuggingFaceModel, HuggingFacePredictor
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)


In [22]:
!pygmentize globals.py

"""
Global variables used throughout the code.
"""
import os
import boto3
import sagemaker

# model deployment
HF_MODEL_ID = "anymodality/llava-v1.5-13b"
HF_MODEL_ID: str = "anymodality/llava-v1.5-7b"

HF_TASK: str = "question-answering"
TRANSFORMERS_VERSION: str = "4.28.1"
PYTORCH_VERSION: str = "2.0.0"
PYTHON_VERSION: str = "py310"

# S3 bucket strucutre, we use the default sagemaker bucket in the current region
BUCKET_NAME: str = sagemaker.Session().default_bucket()
BUCKET_PREFIX: str = "multimodal"
BUCKET_EMB_PREFIX: str = f"{BUCKET_PREFIX}/osi-embeddings-json"
BUCKET_IMG_PREFIX: str = f"{BUCKET_PREFIX}/img"

# Amazon Titan multimodal model
AWS_REGION: str = boto3.Session().region_name
FMC_URL: str = f"https://bedrock-runtime.{AWS_REGION}.amazonaws.com"
FMC_MODEL_ID: str = "amazon.titan-embed-image-v1"
ACCEPT_ENCODING: str = "application/json"
CONTENT_ENCODING: str = "application/json"

# model.tar.gz path in S3
S3_MODEL_URI: str = os.path.join("s3://", BUCKET_NAME, BUCKET_PREFIX, 

In [23]:
# s3 = boto3.client('s3')


outputs = get_cfn_outputs(g.CFN_STACK_NAME)
host = outputs['MultimodalCollectionEndpoint'].split('//')[1]
session = boto3.Session()
credentials = session.get_credentials()
auth = AWSV4SignerAuth(credentials, g.AWS_REGION, g.OS_SERVICE)

os_client = OpenSearch(
    hosts = [{'host': host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize = 20
)


[2024-01-07 00:08:25,740] p26314 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [24]:
endpoint_name = Path(g.ENDPOINT_FILENAME).read_text()

predictor = HuggingFacePredictor(endpoint_name)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [25]:
def get_text_embeddings(bedrock: botocore.client, image: str) -> np.ndarray:
    body = json.dumps(dict(inputText=image))
    try:
        response = bedrock.invoke_model(
            body=body, modelId=g.FMC_MODEL_ID, accept=g.ACCEPT_ENCODING, contentType=g.CONTENT_ENCODING
        )
        response_body = json.loads(response.get("body").read())
        embeddings = np.array([response_body.get("embedding")]).astype(np.float32)
    except Exception as e:
        logger.error(f"exception while text={text}, exception={e}")
        embeddings = None

    return embeddings



In [26]:
bedrock = boto3.client(service_name="bedrock-runtime", endpoint_url=g.FMC_URL)

In [27]:
search_text = "What is the third step in the AI/ML fly wheel"
text_embeddings = get_text_embeddings(bedrock, search_text)

In [28]:
query = {
    "size": 1,
    "query": {
        "knn": {
            "vector_embedding": {
                "vector": text_embeddings[0].tolist(),
                "k": 1
            }
        }
    }
}
index = outputs['OpenSearchIndexName']
try:
    image_based_search_response = os_client.search(body=query, index=index)
except Exception as ex:
    print(ex)

[2024-01-07 00:08:26,289] p26314 {base.py:259} INFO - POST https://jcbl0nhke4fyxk2bfjz2.us-east-1.aoss.amazonaws.com:443/multimodalslidesindex/_search [status:200 request:0.291s]


In [29]:
s3_img_path = image_based_search_response.get('hits', {}).get('hits')[0].get('_source').get('image_path')

In [30]:
print(f"going to analyze image \"{s3_img_path}\"")
data = {
    "image" : s3_img_path,
    "question" : search_text,
    "temperature" : 0.1,
}

output = predictor.predict(data)
print(f"Image=\"{s3_img_path}\"\nDescription: {output}\n\n")

going to analyze image "s3://sagemaker-us-east-1-015469603702/multimodal/img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_12.jpg"
Image="s3://sagemaker-us-east-1-015469603702/multimodal/img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_12.jpg"
Description: The third step in the AI/ML fly wheel is "Investment." This refers to the financial resources and investments required to develop and maintain AI/ML systems. These investments can include funding for research and development, infrastructure, and personnel. Investment is crucial for the growth and success of AI/ML initiatives, as it enables organizations to acquire the necessary tools, technologies, and expertise to build and improve their AI/ML capabilities.


